In [1]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'

# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

# separate out those links that begin with four digits 
file_list = [x for x in link_list if str.isdigit(x[0:4])]

file_list


['20181030.export.CSV.zip',
 '20181029.export.CSV.zip',
 '20181028.export.CSV.zip',
 '20181027.export.CSV.zip',
 '20181026.export.CSV.zip',
 '20181025.export.CSV.zip',
 '20181024.export.CSV.zip',
 '20181023.export.CSV.zip',
 '20181022.export.CSV.zip',
 '20181021.export.CSV.zip',
 '20181020.export.CSV.zip',
 '20181019.export.CSV.zip',
 '20181018.export.CSV.zip',
 '20181017.export.CSV.zip',
 '20181016.export.CSV.zip',
 '20181015.export.CSV.zip',
 '20181014.export.CSV.zip',
 '20181013.export.CSV.zip',
 '20181012.export.CSV.zip',
 '20181011.export.CSV.zip',
 '20181010.export.CSV.zip',
 '20181009.export.CSV.zip',
 '20181008.export.CSV.zip',
 '20181007.export.CSV.zip',
 '20181006.export.CSV.zip',
 '20181005.export.CSV.zip',
 '20181004.export.CSV.zip',
 '20181003.export.CSV.zip',
 '20181002.export.CSV.zip',
 '20181001.export.CSV.zip',
 '20180930.export.CSV.zip',
 '20180929.export.CSV.zip',
 '20180928.export.CSV.zip',
 '20180927.export.CSV.zip',
 '20180926.export.CSV.zip',
 '20180925.export.CS

In [ ]:
import os.path
import urllib
import zipfile
import glob
import operator

local_path = './gdelt_data/'

fips_country_code = 'ID'
infilecounter = 0
outfilecounter = 0

for compressed_file in file_list[:2125]:
    print(compressed_file),
    
    # if we dont have the compressed file stored locally, go get it. Keep trying if necessary.
    while not os.path.isfile(local_path+compressed_file): 
        print('downloading,'),
        urllib.request.urlretrieve(url=gdelt_base_url+compressed_file, 
                           filename=local_path+compressed_file)
        
    # extract the contents of the compressed file to a temporary directory    
    print('extracting,'),
    z = zipfile.ZipFile(file=local_path+compressed_file, mode='r')    
    z.extractall(path=local_path+'tmp/')
    
    # parse each of the csv files in the working directory, 
    print('parsing,'),
    for infile_name in glob.glob(local_path+'tmp/*'):
        outfile_name = local_path+'country/'+fips_country_code+'/'+fips_country_code+'%04i.tsv'%outfilecounter
        
        # open the infile and outfile
        with open(infile_name, mode='r') as infile, open(outfile_name, mode='w') as outfile:
            for line in infile:
                # extract lines with our interest country code
                if fips_country_code in operator.itemgetter(51, 37, 44)(line.split('\t')):    
                    outfile.write(line)
            outfilecounter +=1
            
        # delete the temporary file
        os.remove(infile_name)
    infilecounter +=1
    print('done')